# danmf

In [ ]:
import numpy as np
import networkx as nx
from typing import List, Dict
from sklearn.decomposition import NMF
from karateclub.estimator import Estimator


class DANMF(Estimator):
    r"""An implementation of `"DANMF" <https://smartyfh.com/Documents/18DANMF.pdf>`_
    from the CIKM '18 paper "Deep Autoencoder-like Nonnegative Matrix Factorization for
    Community Detection". The procedure uses telescopic non-negative matrix factorization
    in order to learn a cluster membership distribution over nodes. The method can be
    used in an overlapping and non-overlapping way.

    Args:
        layers (list): Autoencoder layer sizes in a list of integers. Default [32, 8].
        pre_iterations (int): Number of pre-training epochs. Default 100.
        iterations (int): Number of training epochs. Default 100.
        seed (int): Random seed for weight initializations. Default 42.
        lamb (float): Regularization parameter. Default 0.01.
        seed (int): Random seed value. Default is 42.
    """

    def __init__(
        self,
        layers: List[int] = [32, 8],
        pre_iterations: int = 100,
        iterations: int = 100,
        seed: int = 42,
        lamb: float = 0.01,
    ):
        self.layers = layers
        self.pre_iterations = pre_iterations
        self.iterations = iterations
        self.seed = seed
        self.lamb = lamb
        self._p = len(self.layers)
        self.seed = seed

    def _setup_target_matrices(self, graph):
        """
        Setup target matrix for pre-training process.

        Arg types:
            * **graph** *(NetworkX graph)* - The graph being clustered.
        """
        self._graph = graph
        self._A = nx.adjacency_matrix(
            self._graph, nodelist=range(self._graph.number_of_nodes())
        )
        self._L = nx.laplacian_matrix(
            self._graph, nodelist=range(self._graph.number_of_nodes())
        )
        self._D = self._L + self._A

    def _setup_z(self, i):
        """
        Setup target matrix for pre-training process.

        Arg types:
            * **i** *(int)* - The layer index.
        """
        if i == 0:
            self._Z = self._A
        else:
            self._Z = self._V_s[i - 1]

    def _sklearn_pretrain(self, i):
        """
        Pre-training a single layer of the model with sklearn.

        Arg types:
            * **i** *(int)* - The layer index.
        """
        nmf_model = NMF(
            n_components=self.layers[i],
            init="random",
            random_state=self.seed,
            max_iter=self.pre_iterations,
        )

        U = nmf_model.fit_transform(self._Z)
        V = nmf_model.components_
        return U, V

    def _pre_training(self):
        """
        Pre-training each NMF layer.
        """
        self._U_s = []
        self._V_s = []
        for i in range(self._p):
            self._setup_z(i)
            U, V = self._sklearn_pretrain(i)
            self._U_s.append(U)
            self._V_s.append(V)

    def _setup_Q(self):
        """
        Setting up Q matrices.
        """
        self._Q_s = [None for _ in range(self._p + 1)]
        self._Q_s[self._p] = np.eye(self.layers[self._p - 1])
        for i in range(self._p - 1, -1, -1):
            self._Q_s[i] = np.dot(self._U_s[i], self._Q_s[i + 1])

    def _update_U(self, i):
        """
        Updating left hand factors.

        Arg types:
            * **i** *(int)* - The layer index.
        """
        if i == 0:
            R = self._U_s[0].dot(self._Q_s[1].dot(self._VpVpT).dot(self._Q_s[1].T))
            R = R + self._A_sq.dot(self._U_s[0].dot(self._Q_s[1].dot(self._Q_s[1].T)))
            Ru = 2 * self._A.dot(self._V_s[self._p - 1].T.dot(self._Q_s[1].T))
            self._U_s[0] = (self._U_s[0] * Ru) / np.maximum(R, 10**-10)
        else:
            R = (
                self._P.T.dot(self._P)
                .dot(self._U_s[i])
                .dot(self._Q_s[i + 1])
                .dot(self._VpVpT)
                .dot(self._Q_s[i + 1].T)
            )
            R = R + self._A_sq.dot(self._P).T.dot(self._P).dot(self._U_s[i]).dot(
                self._Q_s[i + 1]
            ).dot(self._Q_s[i + 1].T)
            Ru = 2 * self._A.dot(self._P).T.dot(self._V_s[self._p - 1].T).dot(
                self._Q_s[i + 1].T
            )
            self._U_s[i] = (self._U_s[i] * Ru) / np.maximum(R, 10**-10)

    def _update_P(self, i):
        """
        Setting up P matrices.

        Arg types:
            * **i** *(int)* - The layer index.
        """
        if i == 0:
            self._P = self._U_s[0]
        else:
            self._P = self._P.dot(self._U_s[i])

    def _update_V(self, i):
        """
        Updating right hand factors.

        Arg types:
            * **i** *(int)* - The layer index.
        """
        if i < self._p - 1:
            Vu = 2 * self._A.dot(self._P).T
            Vd = self._P.T.dot(self._P).dot(self._V_s[i]) + self._V_s[i]
            self._V_s[i] = self._V_s[i] * Vu / np.maximum(Vd, 10**-10)
        else:
            Vu = (
                2 * self._A.dot(self._P).T + (self.lamb * self._A.dot(self._V_s[i].T)).T
            )
            Vd = self._P.T.dot(self._P).dot(self._V_s[i])
            Vd = Vd + self._V_s[i] + (self.lamb * self._D.dot(self._V_s[i].T)).T
            self._V_s[i] = self._V_s[i] * Vu / np.maximum(Vd, 10**-10)

    def _setup_VpVpT(self):
        self._VpVpT = self._V_s[self._p - 1].dot(self._V_s[self._p - 1].T)

    def _setup_Asq(self):
        self._A_sq = self._A.dot(self._A.T)

    def get_embedding(self) -> np.array:
        r"""Getting the bottleneck layer embedding.

        Return types:
            * **embedding** *(Numpy array)* - The bottleneck layer embedding of nodes.
        """
        embedding = [self._P, self._V_s[-1].T]
        embedding = np.concatenate(embedding, axis=1)
        return embedding

    def get_memberships(self) -> Dict[int, int]:
        r"""Getting the cluster membership of nodes.

        Return types:
            * **memberships** *(dict)*: Node cluster memberships.
        """
        index = np.argmax(self._P, axis=1)
        memberships = {int(i): int(index[i]) for i in range(len(index))}
        return memberships

    def fit(self, graph: nx.classes.graph.Graph):
        """
        Fitting a DANMF clustering model.

        Arg types:
            * **graph** *(NetworkX graph)* - The graph to be clustered.
        """
        self._set_seed()
        graph = self._check_graph(graph)
        self._setup_target_matrices(graph)
        self._pre_training()
        self._setup_Asq()
        for iteration in range(self.iterations):
            self._setup_Q()
            self._setup_VpVpT()
            for i in range(self._p):
                self._update_U(i)
                self._update_P(i)
                self._update_V(i)


# mnmf

In [ ]:
import numpy as np
import networkx as nx
from typing import Dict
from scipy.sparse import coo_matrix
from karateclub.estimator import Estimator


class MNMF(Estimator):
    r"""An implementation of `"M-NMF" <https://aaai.org/ocs/index.php/AAAI/AAAI17/paper/view/14589/13763>`_
    from the AAAI '17 paper "Community Preserving Network Embedding".
    The procedure uses joint non-negative matrix factorization with modularity
    based regularization in order to learn a cluster membership distribution
    over nodes. The method can be used in an overlapping and non-overlapping way.

    Args:
        dimensions (int): Number of dimensions. Default is 128.
        clusters (int): Number of clusters. Default is 10.
        lambd (float): KKT penalty. Default is 0.2
        alpha (float): Clustering penalty. Default is 0.05.
        beta (float): Modularity regularization penalty. Default is 0.05.
        iterations (int): Number of power iterations. Default is 200.
        lower_control (float): Floating point overflow control. Default is 10**-15.
        eta (float): Similarity mixing parameter. Default is 5.0.
        seed (int): Random seed value. Default is 42.
    """

    def __init__(
        self,
        dimensions: int = 128,
        clusters: int = 10,
        lambd: float = 0.2,
        alpha: float = 0.05,
        beta: float = 0.05,
        iterations: int = 200,
        lower_control: float = 10**-15,
        eta: float = 5.0,
        seed: int = 42,
    ):

        self.dimensions = dimensions
        self.clusters = clusters
        self.lambd = lambd
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.lower_control = lower_control
        self.eta = eta
        self.seed = seed

    def _modularity_generator(self):
        """Calculating the sparse modularity matrix."""
        degs = nx.degree(self._graph)
        e_count = self._graph.number_of_edges()
        n_count = self._graph.number_of_nodes()
        modularity_mat_shape = (n_count, n_count)
        indices_1 = np.array(
            [edge[0] for edge in self._graph.edges()]
            + [edge[1] for edge in self._graph.edges()]
        )
        indices_2 = np.array(
            [edge[1] for edge in self._graph.edges()]
            + [edge[0] for edge in self._graph.edges()]
        )
        scores = [
            float(degs[e[0]] * degs[e[1]]) / (2 * e_count) for e in self._graph.edges()
        ]
        scores = scores + [
            float(degs[e[1]] * degs[e[0]]) / (2 * e_count) for e in self._graph.edges()
        ]
        mod_matrix = coo_matrix(
            (scores, (indices_1, indices_2)), shape=modularity_mat_shape
        )
        return mod_matrix

    def _setup_matrices(self):
        """Creating parameter matrices and target matrices."""
        self._number_of_nodes = nx.number_of_nodes(self._graph)
        self._M = np.random.uniform(0, 1, (self._number_of_nodes, self.dimensions))
        self._U = np.random.uniform(0, 1, (self._number_of_nodes, self.dimensions))
        self._H = np.random.uniform(0, 1, (self._number_of_nodes, self.clusters))
        self._C = np.random.uniform(0, 1, (self.clusters, self.dimensions))
        self._B1 = nx.adjacency_matrix(
            self._graph, nodelist=range(self._graph.number_of_nodes())
        )
        self._B2 = self._modularity_generator()
        self._X = np.transpose(self._U)
        overlaps = self._B1.dot(self._B1)
        self._S = self._B1 + self.eta * self._B1 * (overlaps)

    def _update_M(self):
        """Update matrix M."""
        enum = self._S.dot(self._U)
        denom = np.dot(self._M, np.dot(np.transpose(self._U), self._U))
        denom[denom < self.lower_control] = self.lower_control
        self._M = np.multiply(self._M, enum / denom)
        row_sums = self._M.sum(axis=1)
        self._M = self._M / row_sums[:, np.newaxis]

    def _update_U(self):
        """Update matrix U."""
        enum = self._S.dot(self._M) + self.alpha * np.dot(self._H, self._C)
        denom = np.dot(
            self._U,
            np.dot(np.transpose(self._M), self._M)
            + self.alpha * np.dot(np.transpose(self._C), self._C),
        )
        denom[denom < self.lower_control] = self.lower_control
        self._U = np.multiply(self._U, enum / denom)
        row_sums = self._U.sum(axis=1)
        self._U = self._U / row_sums[:, np.newaxis]

    def _update_C(self):
        """Update matrix C."""
        enum = np.dot(np.transpose(self._H), self._U)
        denom = np.dot(self._C, np.dot(np.transpose(self._U), self._U))
        denom[denom < self.lower_control] = self.lower_control
        frac = enum / denom
        self._C = np.multiply(self._C, frac)
        row_sums = self._C.sum(axis=1)
        self._C = self._C / row_sums[:, np.newaxis]

    def _update_H(self):
        """Update matrix H."""
        B1H = self._B1.dot(self._H)
        B2H = self._B2.dot(self._H)
        HHH = np.dot(self._H, (np.dot(np.transpose(self._H), self._H)))
        UC = np.dot(self._U, np.transpose(self._C))
        rooted = np.square(2 * self.beta * B2H) + np.multiply(
            16 * self.lambd * HHH,
            (
                2 * self.beta * B1H
                + 2 * self.alpha * UC
                + (4 * self.lambd - 2 * self.alpha) * self._H
            ),
        )
        rooted[rooted < 0] = 0
        sqroot_1 = np.sqrt(rooted)
        enum = -2 * self.beta * B2H + sqroot_1
        denom = 8 * self.lambd * HHH
        denom[denom < self.lower_control] = self.lower_control
        rooted = enum / denom
        rooted[rooted < 0] = 0
        sqroot_2 = np.sqrt(rooted)
        self._H = np.multiply(self._H, sqroot_2)
        row_sums = self._H.sum(axis=1)
        self._H = self._H / row_sums[:, np.newaxis]

    def get_memberships(self) -> Dict[int, int]:
        r"""Getting the cluster membership of nodes.

        Return types:
            * **memberships** *(dict)* - Node cluster memberships.
        """
        indices = np.argmax(self._H, axis=1)
        memberships = {i: membership for i, membership in enumerate(indices)}
        return memberships

    def get_embedding(self) -> np.array:
        r"""Getting the node embedding.

        Return types:
            * **embedding** *(Numpy array)* - The embedding of nodes.
        """
        embedding = self._U
        return embedding

    def get_cluster_centers(self) -> np.array:
        r"""Getting the node embedding.

        Return types:
            * **centers** *(Numpy array)* - The cluster centers.
        """
        centers = self._C
        return centers

    def fit(self, graph: nx.classes.graph.Graph):
        """
        Fitting an M-NMF clustering model.

        Arg types:
            * **graph** *(NetworkX graph)* - The graph to be clustered.
        """
        self._set_seed()
        graph = self._check_graph(graph)
        self._graph = graph
        self._setup_matrices()
        for _ in range(self.iterations):
            self._update_M()
            self._update_U()
            self._update_C()
            self._update_H()
